# Introduction

## Background

Today, 21 December 2021, we hold a portfolio consisting of physical assets and European options. The specification of each component is as follows:

1. A short position on 1,000 units of the ASX 200
2. A long position on 2,500 units of the S\&P 500 (*Note: prices of the S\&P 500 provided in the data set are in US dollars*)
3. A long position on US\$ 4 million
4. A long position on 10,000 units of a European call option on a share of the ASX 200 with a strike price of \$6,400 per unit and an expiry date of 26 November 2022
5. A long position on 20,000 units of a European put option on a share of the S\&P 500 with a strike price of \$ 2,980 per unit and an expiry date of 23 December 2022

We assume that the risk-free interest rates with maturity 26 November 2022 and 23 December 2022 are 3\% and 3.5\%, respectively. For purposes of option pricing, assume that the (annual) volatility of the ASX 200 and S\&P 500 prices are 20\% and 25\%, respectively. 

## Tasks

Given the time series of financial data in `WorkshopWeek7Data.csv`, perform the following analysis:

1. Determine the per-unit price, delta, and gamma of the European options in the portfolio. What is the resulting position value?
2. Determine the marked-to-market value of the entire portfolio at the valuation date (21 December 2021).
3. Compute the undiversified one-day and 10-day 90\% value-at-risk for the entire portfolio using the delta-gamma-normal approach.
4. Compute the diversified one-day and 10-day 90\% value-at-risk for the entire portfolio using the delta-gamma-normal approach.

## Remarks

1. In this analysis, we shall use the log-returns of the risk factors instead of the percentage return.
2. While the historical time series of S&P 500 prices are denominated in US dollars, assume that the given strike price is in Australian dollars. As such, you may first convert the given S&P 500 prices to Australian dollars using the given time series of USD/AUD exchange rates and use the converted prices for the rest of the analysis. **How would the analysis differ if we do not convert the S&P500 prices and use both the US dollar price and the exchange rate as risk factors?**

# Preparation

## Utilities: European Option Pricing Functions

We load the required libraries and recall the functions we have written to price European options and to calculate their delta and gamma under the Black-Scholes-Merton model.

In [ ]:
# Required libraries
import numpy as np
import scipy as sp
import scipy.stats
import pandas as pd
import math
from datetime import datetime
from datetime import timedelta

In [ ]:
# Pricing function for European calls and puts
def BSprice(spot_price, strike_price, time_to_maturity, risk_free_rate, dividend_yield, volatility, option_type):
    
    # Use mathematical notation for function inputs
    s = spot_price
    K = strike_price
    tau = time_to_maturity # (T-t) in notes
    r = risk_free_rate
    q = dividend_yield
    sigma = volatility
    
    # Calculate d1 and d2
    d1 = (np.log(s / K) + (r - q + 0.5 * sigma ** 2) * tau) / (sigma * np.sqrt(tau))
    d2 = d1 - sigma * np.sqrt(tau)
    
    # Calculate option prices
    if option_type == 'call':
        price = (np.exp(-q * tau) * s * sp.stats.norm.cdf(d1, 0.0, 1.0) \
                 - np.exp(-r * tau) * K * sp.stats.norm.cdf(d2, 0.0, 1.0))
        
    if option_type == 'put':
        price = (np.exp(-r * tau) * K * sp.stats.norm.cdf(-d2, 0.0, 1.0) \
                 - np.exp(-q * tau) * s * sp.stats.norm.cdf(-d1, 0.0, 1.0))
        
    return price

In [ ]:
# Function to calculate option delta under the Black-Scholes-Merton model
def BSdelta(spot_price, strike_price, time_to_maturity, risk_free_rate, dividend_yield, volatility, option_type):
    
    # Use mathematical notation for function inputs
    s = spot_price
    K = strike_price
    tau = time_to_maturity # (T-t) in notes
    r = risk_free_rate
    q = dividend_yield
    sigma = volatility
    
    # Calculate d1 and d2
    d1 = (np.log(s / K) + (r - q + 0.5 * sigma ** 2) * tau) / (sigma * np.sqrt(tau))
    
    # Calculate option delta
    if option_type == 'call':
        value = sp.stats.norm.cdf(d1, 0.0, 1.0)
    
    if option_type == 'put':
        value = sp.stats.norm.cdf(d1, 0.0, 1.0) - 1
    
    return value    

In [ ]:
# Function to calculate option gamma under the Black-Scholes-Merton model
def BSgamma(spot_price, strike_price, time_to_maturity, risk_free_rate, dividend_yield, volatility):
    
    # Use mathematical notation for function inputs
    s = spot_price
    K = strike_price
    tau = time_to_maturity # (T-t) in notes
    r = risk_free_rate
    q = dividend_yield
    sigma = volatility
    
    # Compute option gamma (same for calls and puts)
    d1 = (np.log(s / K) + (r - q + 0.5 * sigma ** 2) * tau) / (sigma * np.sqrt(tau))
    value = sp.stats.norm.pdf(d1, 0.0, 1.0) / (s * sigma * np.sqrt(tau))
    
    return value

## Data Pre-processing

We load the given historical data in `WorkshopWeek7Data.csv`, express the exchange rates as XX AUD per USD, and compute the Australian dollar price of the S\&P 500.

In [ ]:
# Load data
fin_data = pd.read_csv('WorkshopWeek7Data.csv', index_col = 0)

# Write exchange rates as XX AUD per USD
fin_data['FXrate'] = 1 / fin_data['USDperAUD']

# Convert the S&P 500 price to Australian dollars
fin_data['SPXComp'] = fin_data['SPXCompUS'] * fin_data['FXrate']

fin_data.head()

In [ ]:
# Extract market data on the valuation date
fin_data_today = fin_data.loc['2021-12-21']

fin_data_today

In [ ]:
# Compute the log-returns of all market variables
fin_data_logret = np.log(fin_data).diff().tail(-1)

fin_data_logret.head()

In [ ]:
# Compute the covariance matrix of all returns time series
fin_data_logret_cov = fin_data_logret.cov()

fin_data_logret_cov

# Mark-to-Market Valuation

## European Options

We encode all relevant information for the pricing of the European options in the portfolio, we also need to compute: 

1. The time to maturity in years (for simplicity, also assume 252 trading days when converting the time difference in days to years, although there is no reason there shouldn't be a different assumption for this conversion)
2. ~~The annual volatility of the underlying price processes (i.e. the annualized standard deviation of the log-returns) from the given data.~~ (It can be verified that using the volatility implied by the historical time series of prices will lead to very small option prices).

In [ ]:
# Given option parameters
strike_ASX = 6400
strike_SPX = 2980
r_ASX = 0.03
r_SPX = 0.035
vol_ASX = 0.20
vol_SPX = 0.25
N_op_ASX = 10000
N_op_SPX = 20000


# Compute time to maturity
date_today = datetime(2021, 12, 21)
expdate_ASX = datetime(2022, 11, 26)
expdate_SPX = datetime(2022, 12, 23)

mat_ASX = (expdate_ASX - date_today).days / 252
mat_SPX = (expdate_SPX - date_today).days / 252

# # Compute the annual volatilities in the underlying asset prices
# vol_ASX = np.sqrt(fin_data_logret_cov.loc['ASX200', 'ASX200'] / 252)
# vol_SPX = np.sqrt(fin_data_logret_cov.loc['SPXComp', 'SPXComp'] / 252)

We are now ready to calculate the option price, delta, and gamma. Note that the quantities below are *per unit* of the option. It is only when we calculate the value of the position that we multiply the option prices by the number of units.

In [ ]:
# Option prices
price_ASX = BSprice(fin_data_today['ASX200'], strike_ASX, mat_ASX, r_ASX, 0, vol_ASX, 'call')
price_SPX = BSprice(fin_data_today['SPXComp'], strike_SPX, mat_SPX, r_SPX, 0, vol_SPX, 'put')

(price_ASX, price_SPX)

In [ ]:
# Option deltas
delta_ASX = BSdelta(fin_data_today['ASX200'], strike_ASX, mat_ASX, r_ASX, 0, vol_ASX, 'call')
delta_SPX = BSdelta(fin_data_today['SPXComp'], strike_SPX, mat_SPX, r_SPX, 0, vol_SPX, 'put')

(delta_ASX, delta_SPX)

In [ ]:
# Option gammas
gamma_ASX = BSgamma(fin_data_today['ASX200'], strike_ASX, mat_ASX, r_ASX, 0, vol_ASX)
gamma_SPX = BSgamma(fin_data_today['SPXComp'], strike_SPX, mat_SPX, r_SPX, 0, vol_SPX)

(gamma_ASX, gamma_SPX)

In [ ]:
# Position values
V_op_ASX = N_op_ASX * price_ASX
V_op_SPX = N_op_SPX * price_SPX

(V_op_ASX, V_op_SPX)

## Physical Assets

The mark-to-market valuation of the physical assets is straightforward, as shown below.

In [ ]:
# Number of units (negative for short position)
N_ASX = -1000
N_SPX = 2000
N_USD = 4000000

# Valuation
V_ASX = N_ASX * fin_data_today['ASX200']
V_SPX = N_SPX * fin_data_today['SPXComp']
V_USD = N_USD * fin_data_today['FXrate']

(V_ASX, V_SPX, V_USD)

## Total Portfolio Value

In [ ]:
# Total marked-to-market value of the portfolio
V_portfolio = V_ASX + V_SPX + V_USD + V_op_ASX + V_op_SPX

V_portfolio

# Risk Factor Mapping

For the physical assets, the relevant risk factors are their respective returns, namely the log-returns of the ASX 200, the log-returns of the (Australian price) of the S\&P 500, and the log-returns of the AUD-per-USD exchange rate. For these assets, we shall use a delta-normal model for the VaR,

\begin{align*}
\Delta V_{\text{ASX}} & = N_{\text{ASX}} \cdot S_{\text{ASX}} \cdot \Delta \ln S_{\text{ASX}} = V_{\text{ASX}} \cdot \Delta \ln S_{\text{ASX}} \\
\Delta V_{\text{SPX}} & = N_{\text{SPX}} \cdot S_{\text{SPX}} \cdot \Delta \ln S_{\text{SPX}} = V_{\text{SPX}} \cdot \Delta \ln S_{\text{SPX}}\\
\Delta V_{\text{USD}} & = N_{\text{USD}} \cdot S_{\text{USD}} \cdot \Delta \ln S_{\text{USD}} = V_{\text{USD}} \cdot \Delta \ln S_{\text{USD}}
\end{align*}

For the European options, we use a delta-gamma-normal model for the VaR. This entails mapping the changes in the option value to the log-return and squared-log-return of the underlying asset,

\begin{align*}
\Delta V_{\text{opASX}} & = N_{\text{opASX}} \left( S_{\text{ASX}} \cdot \frac{\partial c_{\text{ASX}}}{\partial S_{\text{ASX}}} \cdot \Delta \ln S_{\text{ASX}} + \frac{1}{2} S_{\text{ASX}}^2 \cdot \frac{\partial^2 c_{\text{ASX}}}{\partial S_{\text{ASX}}^2} \cdot (\Delta \ln S_{\text{ASX}})^2 \right)\\
\Delta V_{\text{opSPX}} & = N_{\text{opSPX}} \left( S_{\text{SPX}} \cdot \frac{\partial p_{\text{SPX}}}{\partial S_{\text{SPX}}} \cdot \Delta \ln S_{\text{SPX}} + \frac{1}{2} S_{\text{SPX}}^2 \cdot \frac{\partial^2 p_{\text{SPX}}}{\partial S_{\text{SPX}}^2} \cdot (\Delta \ln S_{\text{SPX}})^2 \right)
\end{align*}

Here, $c_{\text{ASX}}$ (resp. $p_{\text{SPX}}$) is the per-unit price of the European call option (resp. put option) on the ASX (resp. SPX). The bracketed expressions are the per-unit change in the European option prices mapped to the log-returns and squared-log-returns of the underlying asset.

The calculation below constructs the time series of risk factors (log-returns and squared-log returns of the ASX 200 and the S\&P 500 and the log-returns of the AUD-per-USD exchange rate) and the associated covariance matrix.

In [ ]:
# Time series of risk factors
RF = fin_data_logret.loc[:, ('ASX200', 'SPXComp', 'FXrate')]
RF['sqASX200'] = RF['ASX200'] ** 2
RF['sqSPXComp'] = RF['SPXComp'] ** 2

RF.head()

In [ ]:
# Full covariance matrix of risk factors
RF_cov = RF.cov()

RF_cov

# Undiversified VaR

To compute the undiversified VaR of the portfolio, we compute the VaR of each component and add the resulting values. The process of computing the VaR for each of the physical assets is the same as the one followed in last week's Workshop.

In [ ]:
# VaR parameter
conf_level = 0.90

## ASX 200

In [ ]:
# Exposure to and (co)variance of risk factor(s)
ASX_exp = V_ASX
ASX_rf_cov = RF_cov.loc['ASX200', 'ASX200']

# One-day VaR
ASX_VaR = -sp.stats.norm.ppf(1 - conf_level) * np.sqrt((ASX_exp ** 2) * ASX_rf_cov)

ASX_VaR

## S\&P 500

In [ ]:
# Exposure to and (co)variance of risk factor(s)
SPX_exp = V_SPX
SPX_rf_cov = RF_cov.loc['SPXComp', 'SPXComp']

# One-day VaR
SPX_VaR = -sp.stats.norm.ppf(1 - conf_level) * np.sqrt((SPX_exp ** 2) * SPX_rf_cov)

SPX_VaR

## USD

In [ ]:
# Exposure to and (co)variance of risk factor(s)
USD_exp = V_USD
USD_rf_cov = RF_cov.loc['FXrate', 'FXrate']

# One-day VaR
USD_VaR = -sp.stats.norm.ppf(1 - conf_level) * np.sqrt((USD_exp ** 2) * USD_rf_cov)

USD_VaR

## European Call Option on ASX 200

The calculation of the VaR for the European options involve a risk factor vector of length two. In particular, we have the following representation for $\Delta V_{\text{opASX}}$,

$$\Delta V_{\text{opASX}} = \mathbf{W}^\top \mathbf{R}^\ell, 
\quad \text{where} \quad
\mathbf{W} = \left[\begin{array}{c}
N_{\text{opASX}} \cdot S_{\text{ASX}} \cdot \frac{\partial c_{\text{ASX}}}{\partial S_{\text{ASX}}} \\
\frac{1}{2} N_{\text{opASX}} \cdot S_{\text{ASX}}^2 \cdot \frac{\partial^2 c_{\text{ASX}}}{\partial S_{\text{ASX}}^2}
\end{array}\right], 
\quad \text{and} \quad
\mathbf{R}^\ell = \left[\begin{array}{c}
\Delta \ln S_{\text{ASX}} \\
(\Delta \ln S_{\text{ASX}})^2
\end{array}\right]$$

The value-at-risk can then be calculated from the general formula $\text{VaR}_c = -\Phi^{-1}(1-c)\sqrt{\mathbf{W}^\top \Sigma \mathbf{W}}$, where $\Sigma$ is the covariance matrix of $\mathbf{R}^\ell$.

In [ ]:
# Exposure to and (co)variance of risk factor(s)
opASX_exp = np.array([N_op_ASX * fin_data_today['ASX200'] * delta_ASX,
                     0.5 * N_op_ASX * (fin_data_today['ASX200'] ** 2) * gamma_ASX])
opASX_rf_cov = RF_cov.loc[['ASX200', 'sqASX200'], ['ASX200', 'sqASX200']]

# One-day VaR
opASX_variance = np.matmul(opASX_exp.transpose(), np.matmul(opASX_rf_cov, opASX_exp))
opASX_VaR = -sp.stats.norm.ppf(1 - conf_level) * np.sqrt(opASX_variance)

opASX_VaR

## European Put Option on S\&P 500

In [ ]:
# Exposure to and (co)variance of risk factor(s)
opSPX_exp = np.array([N_op_SPX * fin_data_today['SPXComp'] * delta_SPX,
                     0.5 * N_op_SPX * (fin_data_today['SPXComp'] ** 2) * gamma_SPX])
opSPX_rf_cov = RF_cov.loc[['SPXComp', 'sqSPXComp'], ['SPXComp', 'sqSPXComp']]

# One-day VaR
opSPX_variance = np.matmul(opSPX_exp.transpose(), np.matmul(opSPX_rf_cov, opSPX_exp))
opSPX_VaR = -sp.stats.norm.ppf(1 - conf_level) * np.sqrt(opSPX_variance)

opSPX_VaR

## Undiversified One-Day VaR

The undiversified one-day 90\% VaR for the portfolio is the sum of the one-day 90\% VaR for each instrument. 

In [ ]:
# Undiversified one-day 90% VaR for the portfolio
VaR_undiversified = ASX_VaR + SPX_VaR + USD_VaR + opASX_VaR + opSPX_VaR

VaR_undiversified

## Undiversified 10-day VaR

The corresponding 10-day VaR can be approximated using time scaling.

In [ ]:
# Undiversified 10-day 90% VaR for the portfolio (time scaling)
VaR_undiversified_10day = np.sqrt(10) * VaR_undiversified

VaR_undiversified_10day

# Diversified VaR

We first identify the portfolio-level exposure to each risk factor in $\mathbf{R}^\ell$, which at the portfolio level is now given by

$$\mathbf{R}^\ell = 
\left[\begin{array}{c}
\Delta \ln S_{\text{ASX}} \\
\Delta \ln S_{\text{SPX}} \\
\Delta \ln S_{\text{USD}} \\
(\Delta \ln S_{\text{ASX}})^2 \\
(\Delta \ln S_{\text{SPX}})^2
\end{array}\right].$$

The portfolio-level exposures are given by

$$\mathbf{W} =
\left[\begin{array}{c}
V_{\text{ASX}} + N_{\text{opASX}} \cdot S_{\text{ASX}} \cdot \frac{\partial c_{\text{ASX}}}{\partial S_{\text{ASX}}} \\
V_{\text{SPX}} + N_{\text{opSPX}} \cdot S_{\text{SPX}} \cdot \frac{\partial p_{\text{SPX}}}{\partial S_{\text{SPX}}} \\
V_{\text{USD}} \\
\frac{1}{2} N_{\text{opASX}} \cdot S_{\text{ASX}}^2 \cdot \frac{\partial^2 c_{\text{ASX}}}{\partial S_{\text{ASX}}^2} \\
\frac{1}{2} N_{\text{opSPX}} \cdot S_{\text{SPX}}^2 \cdot \frac{\partial^2 p_{\text{SPX}}}{\partial S_{\text{SPX}}^2}
\end{array}\right].$$

The diversified VaR then follows from the general equation $\text{VaR}_c = -\Phi^{-1}(1 - c) \sqrt{\mathbf{W}^\top \Sigma \mathbf{W}}$.

In [ ]:
# Portfolio-level exposures
portfolio_exp = np.array([V_ASX + N_op_ASX * fin_data_today['ASX200'] * delta_ASX,
                         V_SPX + N_op_SPX * fin_data_today['SPXComp'] * delta_SPX,
                         V_USD,
                        0.5 * N_op_ASX * (fin_data_today['ASX200'] ** 2) * gamma_ASX,
                         0.5 * N_op_SPX * (fin_data_today['SPXComp'] ** 2) * gamma_SPX])

# Portfolio variance
portfolio_variance = np.matmul(portfolio_exp.transpose(),
                              np.matmul(RF_cov, portfolio_exp))

# Diversified one-day VaR
VaR_diversified = -sp.stats.norm.ppf(1 - conf_level) * np.sqrt(portfolio_variance)

VaR_diversified

In [ ]:
# Diversified 10-day VaR
VaR_diversified_10day = np.sqrt(10) * VaR_diversified

VaR_diversified_10day